Импорт библиотек и чтение файла

In [3]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random
import pandas as pd
from scipy import stats

with open('Москва_2021.txt', 'r') as file:
    ages = list(map(int, file.readlines()))

Задание 1 (а). Проверить нулевую гипотезу о нормальном распределении случайной величины «возраст»

In [4]:
# Разбиение данных на 7 равных групп
num_bins = 7
bins = np.linspace(start=int(min(ages)), stop=int(max(ages)), num=num_bins + 1, dtype=int)
print(f'Интервалы 7-ми групп: {bins}')

# Вычисление эмпирических частот для каждой группы
empirical_freq, _ = np.histogram(ages, bins=bins)
print(f'Эмпирические частоты групп: {empirical_freq}')

# Вычисление теоретических частот для каждой группы

# Вычисление середины интервалов групп
avg_bins = []
for i in range(0, len(bins)-1):
  avg_bins.append((bins[i]+bins[i+1])/2)
print(f'Середины интервалов групп: {avg_bins}')

# Вычисление выборочной средней
theoretical_mu=0
for i in range(len(avg_bins)-1):
  theoretical_mu += avg_bins[i] * empirical_freq[i] / len(ages)
print(f'Выборочная средняя: {theoretical_mu}')

# Вычисление выборочного среднего квадратического отклонения
sum1 = 0
for x, n in zip(avg_bins, empirical_freq):
  sum1 += n * (x - theoretical_mu)**2
theoretical_sigma = math.sqrt(sum1 / len(ages))
print(f'Выборочное среднее квадратическое отклонение: {theoretical_sigma}')

# Нормирование границ интервалов
norm_bins = []
for i in range(len(bins)):
  norm_bins.append(round(float((bins[i] - theoretical_mu)/theoretical_sigma), 2))
norm_bins[0] = -math.inf
norm_bins[-1] = math.inf
print(f'Нормированные интервалы: {norm_bins}')

# Вычисление функций Лапласа для каждой границы интервалов
fi = []
for i in range(len(norm_bins)):
  if (norm_bins[i] == -math.inf):
    fi.append(-0.5)
  elif (norm_bins[i] == math.inf):
    fi.append(0.5)
  else:
     fi.append(round(float(stats.norm.cdf(norm_bins[i]) - 0.5), 2))
print(f'Значение функции Лапласа на интервалах: {fi}')

# Вычисление вероятностей Pi попадания X в интервалы и сразу же нахождение теоретических частот
theoretical_freq = []
for i in range(len(fi) - 1):
   theoretical_freq.append(len(ages) * (fi[i+1] - fi[i]))
print(f'Теоретические частоты: {theoretical_freq}')

# Вычисление хи-квадрат наблюдаемого
chi2_stat = 0
for i in range(num_bins):
  chi2_stat += ( ( empirical_freq[i] - theoretical_freq[i] )**2 ) / ( theoretical_freq[i] )

# Вычисление критического значения
alpha = 0.05
critical_value = stats.chi2.ppf(alpha, df=num_bins - 1)

# Вывод результатов
print(f"Критерий хи-квадрат наблюдаемый: {chi2_stat}")
print(f"Критическое точка: {critical_value} \n")

# Проверка гипотезы
if chi2_stat > critical_value:
    print("Отвергаем нулевую гипотезу: данные не распределены нормально.")
else:
    print("Не отвергаем нулевую гипотезу: данные распределены нормально.")

Интервалы 7-ми групп: [14 22 30 39 47 56 64 73]
Эмпирические частоты групп: [3867 8571 7674 6452 3583 1362  914]
Середины интервалов групп: [np.float64(18.0), np.float64(26.0), np.float64(34.5), np.float64(43.0), np.float64(51.5), np.float64(60.0), np.float64(68.5)]
Выборочная средняя: 33.953844493106736
Выборочное среднее квадратическое отклонение: 12.633192165520528
Нормированные интервалы: [-inf, -0.95, -0.31, 0.4, 1.03, 1.75, 2.38, inf]
Значение функции Лапласа на интервалах: [-0.5, -0.33, -0.12, 0.16, 0.35, 0.46, 0.49, 0.5]
Теоретические частоты: [5511.91, 6808.830000000001, 9078.44, 6160.369999999999, 3566.5300000000016, 972.689999999999, 324.2300000000003]
Критерий хи-квадрат наблюдаемый: 2406.6996660929717
Критическое точка: 1.6353828943279067 

Отвергаем нулевую гипотезу: данные не распределены нормально.


(Код из Лабораторной работы №2 - генерация выборок и выборки средних из задания №1)

In [5]:
# Подсчет объема выборок
y = 0.95
# Φ(t) = 0.95/2 = 0,4750. По функции Лапласа:
t = 1.96
N = 32423
d = 3
D = 144.9169680648149

n = math.ceil(t**2*D/d**2)

print(f'Объем выборок: {n}')

# Функция для вычисления выборочной средней
def calculate_average(selection):
    total = 0
    for age in selection:
        total += age
    return total / len(selection)

# Генерация выборок и расчет выборочной средней
averages = []
selections = []
for _ in range(36):
    selection = random.choices(ages, k=n)
    selections.append(selection)
    average = calculate_average(selection)
    averages.append(average)

# Результаты
print(f'Выборочные средние: {averages}')
print(f'Выборки: {selections}')

# Сортируем массив averages и берем левую, правую границы
sortedAverages = sorted(averages)
leftBorder = math.floor(min(averages))
rightBorder = math.ceil(max(averages))
print(f'Правая граница: {rightBorder}')
print(f'Левая граница: {leftBorder}')

# Создаем массив значений интервалов
intervals1 = np.arange(leftBorder, rightBorder + 1)
intervals = []
for i in range(len(intervals1)):
    intervals.append(int(intervals1[i]))
empirical_freq_avg, _ = np.histogram(sortedAverages, bins = intervals)
print(f'Интервалы средних:{intervals}')
print(f'Частота средних на интервалах: {empirical_freq_avg}')

Объем выборок: 62
Выборочные средние: [33.935483870967744, 32.04838709677419, 34.516129032258064, 32.25806451612903, 35.274193548387096, 32.516129032258064, 35.04838709677419, 32.87096774193548, 35.75806451612903, 37.935483870967744, 34.24193548387097, 31.951612903225808, 32.46774193548387, 35.03225806451613, 34.87096774193548, 37.67741935483871, 35.91935483870968, 36.435483870967744, 36.516129032258064, 34.645161290322584, 35.435483870967744, 33.306451612903224, 37.403225806451616, 36.95161290322581, 33.38709677419355, 34.935483870967744, 35.08064516129032, 35.0, 36.096774193548384, 38.20967741935484, 34.983870967741936, 33.225806451612904, 34.274193548387096, 36.82258064516129, 36.25806451612903, 33.0]
Выборки: [[47, 49, 28, 44, 24, 33, 48, 48, 29, 18, 34, 19, 26, 49, 20, 22, 38, 25, 35, 47, 49, 18, 40, 41, 65, 31, 40, 19, 37, 25, 48, 31, 25, 42, 22, 31, 29, 24, 25, 43, 36, 36, 17, 45, 49, 40, 27, 41, 27, 45, 19, 24, 40, 23, 43, 42, 45, 37, 38, 23, 21, 18], [46, 32, 21, 23, 31, 34, 2

Задание 1 (б). Проверить нулевую гипотезу о нормальном распределении случайной величины «средний возраст»

In [6]:
# Вычисление теоретических частот для каждой группы

# Вычисление середины интервалов групп
avg_intervals = []
for i in range(len(intervals)-1):
  avg_intervals.append(float((intervals[i]+intervals[i+1])/2))
print(f'Середины интервалов групп: {avg_intervals}')

# Вычисление выборочной средней
avg_theoretical_mu=0
for i in range(len(avg_intervals)-1):
  avg_theoretical_mu += avg_intervals[i] * empirical_freq_avg[i] / len(sortedAverages)
print(f'Выборочная средняя: {avg_theoretical_mu}')

# Вычисление выборочного среднего квадратического отклонения
sum1 = 0
for x, n in zip(avg_intervals, empirical_freq_avg):
  sum1 += n * (x - avg_theoretical_mu)**2
avg_theoretical_sigma = math.sqrt(sum1 / len(sortedAverages))
print(f'Выборочное среднее квадратическое отклонение: {avg_theoretical_sigma}')

# Нормирование границ интервалов
norm_intervals = []
for i in range(len(intervals)):
  norm_intervals.append(round(float((intervals[i] - avg_theoretical_mu)/avg_theoretical_sigma), 2))
norm_intervals[0] = -math.inf
norm_intervals[-1] = math.inf
print(f'Нормированные интервалы: {norm_intervals}')

# Вычисление функций Лапласа для каждой границы интервалов
avg_fi = []
for i in range(len(norm_intervals)):
  if (norm_intervals[i] == -math.inf):
    avg_fi.append(-0.5)
  elif (norm_intervals[i] == math.inf):
    avg_fi.append(0.5)
  else:
     avg_fi.append(round(float(stats.norm.cdf(norm_intervals[i]) - 0.5), 2))
print(f'Значение функции Лапласа на интервалах: {avg_fi}')

# Вычисление вероятностей Pi попадания X в интервалы и сразу же нахождение теоретических частот
avg_theoretical_freq = []
for i in range(len(avg_fi) - 1):
   avg_theoretical_freq.append(len(sortedAverages) * (avg_fi[i+1] - avg_fi[i]))
print(f'Теоретические частоты: {avg_theoretical_freq}')

# Вычисление хи-квадрат наблюдаемого
avg_chi2_stat = 0
for i in range(len(intervals)-1):
  avg_chi2_stat += ( ( empirical_freq_avg[i] - avg_theoretical_freq[i] )**2 ) / ( avg_theoretical_freq[i] )

# Вычисление критического значения
alpha = 0.05
avg_critical_value = stats.chi2.ppf(alpha, df=len(intervals) - 1 - 1)

# Вывод результатов
print(f"Критерий хи-квадрат наблюдаемый: {avg_chi2_stat}")
print(f"Критическое точка: {avg_critical_value} \n")

# Проверка гипотезы
if avg_chi2_stat > critical_value:
    print("Отвергаем нулевую гипотезу: данные не распределены нормально.")
else:
    print("Не отвергаем нулевую гипотезу: данные распределены нормально.")

Середины интервалов групп: [31.5, 32.5, 33.5, 34.5, 35.5, 36.5, 37.5, 38.5]
Выборочная средняя: 33.84722222222222
Выборочное среднее квадратическое отклонение: 1.9994694512344404
Нормированные интервалы: [-inf, -0.92, -0.42, 0.08, 0.58, 1.08, 1.58, 2.08, inf]
Значение функции Лапласа на интервалах: [-0.5, -0.32, -0.16, 0.03, 0.22, 0.36, 0.44, 0.48, 0.5]
Теоретические частоты: [6.4799999999999995, 5.76, 6.84, 6.84, 5.039999999999999, 2.880000000000001, 1.4399999999999993, 0.7200000000000006]
Критерий хи-квадрат наблюдаемый: 12.150613803026085
Критическое точка: 2.167349909298058 

Отвергаем нулевую гипотезу: данные не распределены нормально.


Задание №2 (а). По двум выборкам из ЛР №2 проверить нулевую гипотезу о равенстве дисперсий генеральных совокупностей при уровне значимости 0,05 при конкурирующей гипотезе 𝐻1: 𝐷1 > 𝐷2

In [17]:
# Выборки 1 и 2 соответственно: selections[0] и selections[1]
print(f'Первая выборка:\n {selections[0]}\n Средняя выборочная: {averages[0]}')
print(f'Вторая выборка:\n {selections[1]}\n Средняя выборочная: {averages[1]}')

# Критическое значение Фишера для k = 61 и a = 0.05
critical_fisher = 1.53

# Вычисление дисперсии для двух выборок
s1 = 0
for x in selections[1]:
  s1 += (x - averages[1])**2
s1 = s1 / len(ages)

s2 = 0
for x in selections[0]:
  s2 += (x - averages[0])**2
s2 = s2 / len(ages)

fisher = s1/s2
print(f'Оценка дисперсии первой выборки: {s1}')
print(f'Оценка дисперсии второй выборки: {s2}')
print(f'Критерий фишер: {fisher}')
print(f'Критическое значение фишера: {critical_fisher} \n')

if fisher < critical_fisher:
  print("Не отвергаем нулевую гипотезу: дисперсии равны.")
else:
  print("Отвергаем нулевую гипотезу: дисперсии не равны.")

Первая выборка:
 [47, 49, 28, 44, 24, 33, 48, 48, 29, 18, 34, 19, 26, 49, 20, 22, 38, 25, 35, 47, 49, 18, 40, 41, 65, 31, 40, 19, 37, 25, 48, 31, 25, 42, 22, 31, 29, 24, 25, 43, 36, 36, 17, 45, 49, 40, 27, 41, 27, 45, 19, 24, 40, 23, 43, 42, 45, 37, 38, 23, 21, 18]
 Средняя выборочная: 33.935483870967744
Вторая выборка:
 [46, 32, 21, 23, 31, 34, 21, 34, 29, 18, 21, 38, 29, 20, 22, 49, 23, 35, 48, 26, 36, 33, 34, 46, 24, 29, 30, 23, 29, 33, 28, 31, 21, 38, 36, 32, 33, 45, 23, 20, 37, 18, 22, 49, 44, 26, 19, 24, 33, 49, 39, 57, 31, 63, 14, 47, 29, 25, 44, 23, 31, 39]
 Средняя выборочная: 32.04838709677419
Оценка дисперсии первой выборки: 0.2040173592421946
Оценка дисперсии второй выборки: 0.22723813143397809
Критерий фишер: 0.8978130472854638
Критическое значение фишера: 1.53 

Не отвергаем нулевую гипотезу: дисперсии равны.


Задание №2 (б). По двум выборкам из ЛР №2 проверить нулевую гипотезу о равенстве дисперсий генеральных совокупностей при уровне значимости 0,05 при конкурирующей гипотезе 𝐻1:𝐷1 ≠ 𝐷2.

In [16]:
# Выборки 1 и 2 соответственно: selections[0] и selections[1]
print(f'Первая выборка:\n {selections[0]}\n Средняя выборочная: {averages[0]}')
print(f'Вторая выборка:\n {selections[1]}\n Средняя выборочная: {averages[1]}')

# Критическое значение Фишера для k = 61 и a = 0.025
critical_fisher = 1.67

# Вычисление дисперсии для двух выборок
s1 = 0
for x in selections[1]:
  s1 += (x - averages[1])**2
s1 = s1 / len(ages)

s2 = 0
for x in selections[0]:
  s2 += (x - averages[0])**2
s2 = s2 / len(ages)

# Определяем большую и меньшую оценку. Первая - большая, вторая - меньшая
if s1 < s2:
  s1, s2 = s2, s1
  
fisher = s1/s2
print(f'Оценка дисперсии первой выборки: {s1}')
print(f'Оценка дисперсии второй выборки: {s2}')
print(f'Критерий фишер: {fisher}')
print(f'Критическое значение фишера: {critical_fisher} \n')

if fisher < critical_fisher:
  print("Не отвергаем нулевую гипотезу: дисперсии равны.")
else:
  print("Отвергаем нулевую гипотезу: дисперсии не равны.")

Первая выборка:
 [47, 49, 28, 44, 24, 33, 48, 48, 29, 18, 34, 19, 26, 49, 20, 22, 38, 25, 35, 47, 49, 18, 40, 41, 65, 31, 40, 19, 37, 25, 48, 31, 25, 42, 22, 31, 29, 24, 25, 43, 36, 36, 17, 45, 49, 40, 27, 41, 27, 45, 19, 24, 40, 23, 43, 42, 45, 37, 38, 23, 21, 18]
 Средняя выборочная: 33.935483870967744
Вторая выборка:
 [46, 32, 21, 23, 31, 34, 21, 34, 29, 18, 21, 38, 29, 20, 22, 49, 23, 35, 48, 26, 36, 33, 34, 46, 24, 29, 30, 23, 29, 33, 28, 31, 21, 38, 36, 32, 33, 45, 23, 20, 37, 18, 22, 49, 44, 26, 19, 24, 33, 49, 39, 57, 31, 63, 14, 47, 29, 25, 44, 23, 31, 39]
 Средняя выборочная: 32.04838709677419
Оценка дисперсии первой выборки: 0.22723813143397809
Оценка дисперсии второй выборки: 0.2040173592421946
Критерий фишер: 1.1138176294313145
Критическое значение фишера: 1.67 

Не отвергаем нулевую гипотезу: дисперсии равны.
